# Laboratory 1: Property Graphs
### Luis Alfredo Leon Villapún
### Liliia Aliakberova

# Part A.3 Evolving the graph
* * *
In this section we have to modify some parts of the data and explore how adaptable a property graph is when receiving data with a different structure than the original one. An advantage of the schemaless structure is that we can implement updates and evolve the database without the need of rethinking the structure of the schema.

## Creating the connector
As in part A.2, let's first create the connector to handle the messages with Neo4j.

In [2]:
from connector import Neo4jConnector
from getpass import getpass

uri = "neo4j://localhost:7687"
user = "neo4j"
password = getpass("Input your password to connect")
conn = Neo4jConnector(uri, user, password)

Input your password to connect········


## Evolving the model

In this section we will perform the necessary queries to transform the data.

In [13]:
# Generate a csv assigning random affiliations to each author

import json, random
path = "/Users/alfredo.leon/Desktop/SDMLab1/PropertyGraphsLab/neo4jcontainerdata/samples/authors/"
with open(f"{path}authors-sample.jsonl", "r") as authors_file:
    with open(f"{path}affiliations.csv", "w") as affiliations_file:
        affiliations_file.write("authorid,affiliation,afftype\n")
        for author in authors_file:
            author = json.loads(author.strip("\n"))
            affiliations = ['Google', 'Meta', 'NVidia', 'UPC', 'ULB', 'Amazon', 'CentraleSupelec', 'UniPd']
            types = ['company', 'company', 'company', 'university', 'university', 'company', 'university', 'university']
            affs = [(a,t) for a,t in zip(affiliations, types)]
            affiliation, tp = random.choice(affs)
            affiliations_file.write(f"{author['authorid']}, {affiliation}, {tp}\n")

In [14]:
def update_affiliations(conn):
    query = """
        LOAD CSV WITH HEADERS FROM ("file:///samples/authors/affiliations.csv") as row
        MATCH (a: Author{author_id: row.authorid})
        WITH row, a
        MERGE (a)-[:AFFILIATED_TO]->(O: Organization {name: row.affiliation, type: row.afftype})
    """
    session = conn.driver.session()
    response = list(session.run(query))
    session.close()
    print("Success")

In [15]:
update_affiliations(conn)

Success
